## Лабораторная работа 3

### Задача

В вашем распоряжении имеется уже предобработанный и очищенный датасет с фактами покупок абонентами телепередач от компании E-Contenta. 

По доступным вам данным, нужно предсказать вероятность покупки других передач этими, а, возможно, и другими абонентами. При решении задачи запрещено использовать библиотеки pandas, sklearn (кроме sklearn.metrics), xgboost и другие. Если scikit-learn (например, но и другие тоже) обернут в классы Transformer и Estimator, то их можно использовать.

In [1]:
import os
import sys
import json

In [2]:
import os
import sys

os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, FloatType
from pyspark.sql.functions import col, array_contains, mean, pandas_udf, PandasUDFType, split as split_
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import Pipeline

### Данные

In [4]:
!hdfs dfs -ls /labs/slaba03/

Found 4 items
-rw-r--r--   3 hdfs hdfs   91066524 2022-01-06 18:46 /labs/slaba03/laba03_items.csv
-rw-r--r--   3 hdfs hdfs   29965581 2022-01-06 18:46 /labs/slaba03/laba03_test.csv
-rw-r--r--   3 hdfs hdfs   74949368 2022-01-06 18:46 /labs/slaba03/laba03_train.csv
-rw-r--r--   3 hdfs hdfs  871302535 2022-01-06 18:46 /labs/slaba03/laba03_views_programmes.csv


###  laba03_train.csv 


* purchase - факты покупки 
* user_id - id пользователя
* item_id - id телепередачи


In [5]:
schema = StructType() \
      .add("user_id", IntegerType(), True) \
      .add("item_id", IntegerType(), True) \
      .add("purchase", IntegerType(), True)
      
df_user = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("/labs/slaba03/laba03_train.csv")


In [6]:
df_user.show(5)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
|   1654|  89901|       0|
|   1654| 100504|       0|
+-------+-------+--------+
only showing top 5 rows



In [7]:
df_user.groupBy('purchase').count().show()

+--------+-------+
|purchase|  count|
+--------+-------+
|       1|  10904|
|       0|5021720|
+--------+-------+



In [8]:
mean_purchase_user = df_user.groupBy('user_id').agg(mean('purchase').alias('mean_purchase_user'))

###  laba03_test.csv 


* purchase - факты покупки 
* user_id - id пользователя
* item_id - id телепередачи

In [9]:
schema = StructType() \
      .add("user_id", IntegerType(), True) \
      .add("item_id", IntegerType(), True) 
      
      
df_user_test = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("/labs/slaba03/laba03_test.csv")


In [10]:
df_user_test.show(5)

+-------+-------+
|user_id|item_id|
+-------+-------+
|   1654|  94814|
|   1654|  93629|
|   1654|   9980|
|   1654|  95099|
|   1654|  11265|
+-------+-------+
only showing top 5 rows



In [11]:
df_user_test.count()

2156840

###  laba03_items.csv


* item_id - Соответствует item_id в предыдущем файле.
* content_type - тип контента
* title - название передачи, текстовое поле.
* year - год выпуска передачи, число.
* genres - поле с жанрами передачи, разделёнными через запятую.


In [12]:
read_items_schema = StructType(fields=[StructField('item_id', IntegerType()), 
StructField('channel_id', IntegerType()),
StructField('datetime_availability_start', StringType()),
StructField('datetime_availability_stop', StringType()),
StructField('datetime_show_start', StringType()),
StructField('datetime_show_stop', StringType()),
StructField('content_type', IntegerType()),
StructField('title', StringType(), nullable=True),
StructField('year', FloatType(), nullable=True),
StructField('genres', StringType()),
StructField('region_id', IntegerType()),
]) 




df_items = spark.read.format("csv") \
      .option("header", True) \
      .option("sep", "\t")\
      .schema(read_items_schema) \
      .load("/labs/slaba03/laba03_items.csv")

In [13]:
df_items.show(5, False, False)

+-------+----------+---------------------------+--------------------------+-------------------+------------------+------------+--------------------------------------------------------------------------------------+------+-------+---------+
|item_id|channel_id|datetime_availability_start|datetime_availability_stop|datetime_show_start|datetime_show_stop|content_type|title                                                                                 |year  |genres |region_id|
+-------+----------+---------------------------+--------------------------+-------------------+------------------+------------+--------------------------------------------------------------------------------------+------+-------+---------+
|65667  |null      |1970-01-01T00:00:00Z       |2018-01-01T00:00:00Z      |null               |null              |1           |на пробах только девушки (all girl auditions)                                         |2013.0|Эротика|null     |
|65669  |null      |1970-01-01T00:00:00Z

In [14]:
df_items.select('genres').distinct().show()

+--------------------+
|              genres|
+--------------------+
|               Ужасы|
|Драмы,Зарубежные,...|
|      Для детей,Наши|
|Русские мультфиль...|
|Детективы,Триллер...|
|Мистические,Прикл...|
|Приключения,Совет...|
|Западные мультфил...|
|Военные,Приключен...|
|Комедии,Советское...|
|Документальные,За...|
|       Ужасы,Русские|
|Ужасы,Приключения...|
|Юмористические,Ко...|
|Мистические,Ужасы...|
|   Приключения,Драмы|
|Детективы,Драмы,Б...|
|Исторические,Доку...|
|Исторические,Позн...|
|Военные,Приключен...|
+--------------------+
only showing top 20 rows



In [15]:
df_items.select('genres').distinct().count()

1077

In [16]:
df_items = df_items.select('*', split_('genres', ',').alias('first_genre_'))
df_items = df_items.withColumn('first_genre', df_items.first_genre_.getItem(0))

In [17]:
df_items.select('first_genre').distinct().count()

66

In [18]:
df_items.select('first_genre', 'item_id').distinct().groupBy('first_genre').count().show(100)

+--------------------+------+
|         first_genre| count|
+--------------------+------+
|               Ужасы|   336|
|            Анимация|     2|
|           Мелодрама|     3|
|         Мистические|    70|
|        Исторические|     8|
|         Мультфильмы|   241|
|      Документальный|     4|
|          Зарубежные|     4|
|            Семейный|     1|
|Западные мультфильмы|    42|
|              Фильмы|     2|
|      Юмористические|     5|
|                null|    33|
|     Развлекательные|    32|
|                Игры|     7|
|            Передачи|     2|
|     Короткометражки|     2|
| Русские мультфильмы|   169|
|             Боевики|    48|
|           Мелодрамы|    67|
|          Мультфильм|     1|
|           Для детей|    11|
|             Эротика|   121|
|             General|631864|
|    Короткометражные|     5|
|                Наши|     3|
|             Военные|   172|
|          Спортивные|     2|
|             Детские|     5|
|            Семейные|    72|
|      Сов

In [19]:
df_items.select('year').distinct().show()

+------+
|  year|
+------+
|1954.0|
|1951.0|
|1944.0|
|1979.0|
|2010.0|
|1960.0|
|1945.0|
|1924.0|
|1963.0|
|1956.0|
|1938.0|
|1983.0|
|1939.0|
|1999.0|
|1982.0|
|  null|
|2003.0|
|1981.0|
|1916.0|
|1975.0|
+------+
only showing top 20 rows



In [20]:
user_item = df_user.join(df_items, how = 'left', on = 'item_id')

In [21]:
mean_purchase_item = user_item.select('item_id', 'purchase').groupBy('item_id').agg(mean('purchase').alias('mean_purchase_item'))

## laba03_views_programmes.csv

In [22]:
read_users_schema = StructType(fields=[StructField('user_id', IntegerType()), 
StructField('item_id', IntegerType()),
StructField('ts_start', IntegerType()),
StructField('ts_end', IntegerType()),
StructField('item_type', StringType()),
]) 



df_views_programmes = spark.read.format("csv") \
      .option("header", True) \
      .schema(read_users_schema) \
      .load("/labs/slaba03/laba03_views_programmes.csv")

In [23]:
df_views_programmes.show(5, False, False)

+-------+-------+----------+----------+---------+
|user_id|item_id|ts_start  |ts_end    |item_type|
+-------+-------+----------+----------+---------+
|0      |7101053|1491409931|1491411600|live     |
|0      |7101054|1491412481|1491451571|live     |
|0      |7101054|1491411640|1491412481|live     |
|0      |6184414|1486191290|1486191640|live     |
|257    |4436877|1490628499|1490630256|live     |
+-------+-------+----------+----------+---------+
only showing top 5 rows



## Советы

* На качество прогноза в большей степени влияет качество признаков, которые вы сможете придумать из имеющихся данных, нежели выбор и сложность алгоритма.

* Качество входных данных также имеет сильное значение. Существует фраза "garbage in – garbage out". Мусор на входе – мусор на выходе. Потратьте время на подготовку и предобработку данных. Путь к успеху в третьей лабораторной:

* Сосредоточьтесь на формировании следующих фичей: по файлу laba03_train.csv сформируйте признаки, характеризирующие как интенсивно покупает пользователь и "покупаемость" item'ов

* возьмите достаточно мощную модель (например GBTClassifier из pyspark'а)

## Tips 

в датасете очень много категориальных признаков, рассмотрим, как работать с ними

## target encoding

![Cat](https://hsto.org/getpro/habr/upload_files/d85/93e/527/d8593e527edced50d1d22c8e32a05b85.png)

## one hot encoding

![Cat](https://e6v4p8w2.rocketcdn.me/wp-content/uploads/2022/01/One-Hot-Encoding-for-Scikit-Learn-in-Python-Explained-1024x576.png)




# Решение

Первый способ с помощью ALS показал результат 0.74

In [29]:
from pyspark.ml.recommendation import ALS

In [103]:
df_user1 = (df_user
       .withColumnRenamed('user_id','user')
       .withColumnRenamed('item_id', 'item')
       .withColumnRenamed('purchase', 'rating'))

In [105]:
als = ALS(rank=10, maxIter=5, seed=5757)
model = als.fit(df_user1, params={als.coldStartStrategy: "drop"})

In [106]:
predictions = model.transform(df_user_test.withColumnRenamed('user_id','user').withColumnRenamed('item_id', 'item'))

In [107]:
predictions.show(5)

+------+----+-------------+
|  user|item|   prediction|
+------+----+-------------+
|886063|8389|5.0145765E-22|
|900335|8389| 9.762687E-23|
|936359|8389|1.5562003E-23|
|901323|8389|3.6834068E-23|
|928231|8389|1.2087092E-22|
+------+----+-------------+
only showing top 5 rows



In [108]:
predictions = predictions.withColumnRenamed('user', 'user_id').withColumnRenamed('item', 'item_id').withColumnRenamed('prediction', 'purchase')

In [110]:
predictions = predictions.orderBy('user_id', 'item_id')

In [111]:
predictions.show()

+-------+-------+-------------+
|user_id|item_id|     purchase|
+-------+-------+-------------+
|   1654|    336|          0.0|
|   1654|    678|          0.0|
|   1654|    691|          0.0|
|   1654|    696|3.3109543E-23|
|   1654|    763|4.4196544E-24|
|   1654|    795| 7.748142E-22|
|   1654|    861| 6.468716E-23|
|   1654|   1137|1.7891349E-22|
|   1654|   1159|2.2340202E-23|
|   1654|   1428|6.2637883E-22|
|   1654|   1685|6.2442993E-22|
|   1654|   1686|7.3528494E-23|
|   1654|   1704| 1.161839E-22|
|   1654|   2093|          0.0|
|   1654|   2343| 6.340229E-24|
|   1654|   2451|          0.0|
|   1654|   2469| 2.613654E-22|
|   1654|   2603|3.3086378E-24|
|   1654|   2609|          0.0|
|   1654|   2621|8.0559997E-23|
+-------+-------+-------------+
only showing top 20 rows



In [113]:
pandas_pr = predictions.toPandas()

In [116]:
pandas_pr.to_csv('lab03.csv')

Второй способ - бинарная классификация на 2 признаках (target encoding фильма и пользователя)

In [24]:
df_train = df_user.join(mean_purchase_user, on = 'user_id', how = 'left')
df_train = df_train.join(mean_purchase_item, on = 'item_id', how = 'left')

df_test = df_user_test.join(mean_purchase_user, on = 'user_id', how = 'left')
df_test = df_test.join(mean_purchase_item, on = 'item_id', how = 'left')

In [25]:
stringIndexer = StringIndexer(inputCol="first_genre", outputCol="indexed")
model = stringIndexer.fit(df_items)
td = model.transform(df_items)
encoder = OneHotEncoder(inputCol="indexed", outputCol="features")

In [26]:
ohe_genre = encoder.transform(td).select('features', 'item_id')

In [28]:
df_train.show()

+-------+-------+--------+--------------------+--------------------+
|item_id|user_id|purchase|  mean_purchase_user|  mean_purchase_item|
+-------+-------+--------+--------------------+--------------------+
|   8389| 793876|       0|0.001940240589833...|0.005979073243647235|
|   8389| 795620|       0|0.004243827160493827|0.005979073243647235|
|   8389| 851848|       0|3.888024883359253...|0.005979073243647235|
|   8389| 880451|       0|0.009220130618517095|0.005979073243647235|
|   8389| 900203|       0|0.003436426116838488|0.005979073243647235|
|   8389| 920599|       0|0.001556420233463...|0.005979073243647235|
|   8389| 799148|       0|3.878975950349108E-4|0.005979073243647235|
|   8389| 853717|       0|0.001571709233791...|0.005979073243647235|
|   8389| 871607|       0|3.837298541826554E-4|0.005979073243647235|
|   8389| 897570|       0|0.006165703275529865|0.005979073243647235|
|   8389| 895407|       0|0.003097173828881146|0.005979073243647235|
|   8389| 906790|       0|        

In [29]:
assembler = VectorAssembler(inputCols=['mean_purchase_user', 'mean_purchase_item'], outputCol="features")

In [30]:
gbt = GBTClassifier(labelCol='purchase', seed=42)

In [31]:
pipeline = Pipeline(stages=[
    assembler,
    gbt
])

In [32]:
pipeline_model = pipeline.fit(df_train)

In [33]:
predictions = pipeline_model.transform(df_test)

In [34]:
predictions.select('user_id', 'item_id', 'probability', 'prediction').show()

+-------+-------+--------------------+----------+
|user_id|item_id|         probability|prediction|
+-------+-------+--------------------+----------+
| 761341|   8389|[0.95581796033907...|       0.0|
| 776188|   8389|[0.95420734066546...|       0.0|
| 846231|   8389|[0.95378656982703...|       0.0|
| 765780|   8389|[0.95354731831534...|       0.0|
| 814235|   8389|[0.95500291011960...|       0.0|
| 849754|   8389|[0.95569346459518...|       0.0|
| 860986|   8389|[0.95513734150042...|       0.0|
| 887904|   8389|[0.95581796033907...|       0.0|
| 927643|   8389|[0.95378656982703...|       0.0|
| 937273|   8389|[0.95354731831534...|       0.0|
| 903014|   8389|[0.91767024290144...|       0.0|
| 910897|   8389|[0.95581796033907...|       0.0|
| 935644|   8389|[0.95420734066546...|       0.0|
| 854520|   8389|[0.95500291011960...|       0.0|
| 866647|   8389|[0.95513734150042...|       0.0|
| 898782|   8389|[0.95581796033907...|       0.0|
| 788108|   8389|[0.95420734066546...|       0.0|


In [36]:
predictions.groupBy('prediction').count().show()

+----------+-------+
|prediction|  count|
+----------+-------+
|       0.0|2156840|
+----------+-------+



In [40]:
predictions = predictions.withColumnRenamed('probability', 'purchase')
predictions = predictions.select('user_id', 'item_id', 'purchase').orderBy('user_id', 'item_id')

In [43]:
predictions = predictions.orderBy('user_id', 'item_id')

In [44]:
pandas_pr = predictions.toPandas()

In [46]:
pandas_pr['purchase'] = pandas_pr['purchase'].apply(lambda x: x[1])

In [47]:
pandas_pr

,user_id,item_id,purchase
0,1654,336,0.043926
1,1654,678,0.043926
2,1654,691,0.043926
3,1654,696,0.044061
4,1654,763,0.043965
5,1654,795,0.046453
6,1654,861,0.043965
7,1654,1137,0.044272
8,1654,1159,0.044061
9,1654,1428,0.043965


In [48]:
pandas_pr.to_csv('lab03.csv')

In [49]:
spark.stop()